In [64]:
import numpy as np
import pandas as pd

import pytz

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
from astroplan import Observer, FixedTarget

import warnings
warnings.filterwarnings('ignore', category=Warning)


from astroplan.plots import plot_sky, plot_airmass
from astroplan import Observer, FixedTarget, time_grid_from_range, observability_table, moon_illumination
from astroplan import AirmassConstraint, MoonSeparationConstraint

In [66]:
#Creating observing Location
CTIO = Observer(longitude = 70.815 * u.deg,
                       latitude = -30.16527778 * u.deg,
                       elevation = 2788 * u.m,
                       timezone = 'Etc/GMT-4',
                       name = "Cerro Tololo Interamerican Observatory"
                      )

In [67]:
current_time = Time("2019-05-01 12:00:00")
current_time2 = Time("2019-05-15 12:00:00")
midnight_mro = CTIO.midnight(current_time, which='next')
midnight_mro2 = CTIO.midnight(current_time2, which='next')

#Starting range of the RA
CTIO.local_sidereal_time(midnight_mro)

<Longitude 14.58108482 hourangle>

In [68]:
#Ending range of the RA
CTIO.local_sidereal_time(midnight_mro2)

<Longitude 15.4884886 hourangle>

In [69]:
#At local time on when you can observe it.
window_start = Time("2019-05-01 12:00:00")
window_end = Time("2019-05-15 12:00:00")
print(window_start.to_datetime(CTIO.timezone))

2019-05-01 16:00:00+04:00


In [70]:
#Length of observing Period
observe_start = CTIO.twilight_evening_astronomical(window_start, 
                                                   which='nearest')
observe_end = CTIO.twilight_morning_astronomical(window_start, 
                                                 which='next')
observing_range = (observe_end - observe_start)
observe_start
print("You can observe for {0:.1f} tonight".format(observing_range.to(u.h)))

You can observe for 10.4 h tonight


In [77]:
#setting up the catalog
catalog = pd.read_csv('./galaxies.csv', skiprows=58, sep=';',
                      usecols=[2,3,4], names=['Name', 'RA', 'Dec'])

catalog

,Name,RA,Dec
0,2MASX J00011433-7657573,00 01 14.20,-76 57 58.0
1,2MASX J00014012-7450074,00 01 40.60,-74 50 07.5
2,2MASX J00014596-7657144,00 01 45.50,-76 57 13.5
3,2MASX J00031710-7636364,00 03 17.00,-76 36 35.0
4,2MASX J00043598-7523033,00 04 35.55,-75 23 02.0
5,ESO 028-G009,00 05 42.55,-75 42 25.0
6,2MASX J00063721-7500330,00 06 37.70,-75 00 33.0
7,ESO 050-GA005,00 07 37.90,-72 11 57.5
8,2MASX J00094301-7455000,00 09 42.80,-74 55 00.0
9,2MASX J00105352-7627255,00 10 54.10,-76 27 25.5


In [78]:
#narrowed objects via RA
new = catalog[204:214]
new

,Name,RA,Dec
204,WKK 4098,14 35 57.25,-51 55 05.5
205,2MASX J14381221-7239480,14 38 08.90,-72 39 21.0
206,2MASX J14382229-7240149,14 38 22.25,-72 40 12.0
207,WKK 4172,14 42 08.55,-48 53 07.5
208,WKK 4260,14 45 40.10,-49 20 54.5
209,WKK 4266,14 45 49.25,-49 22 48.0
210,WKK 4551,15 01 13.80,-47 59 11.0
211,2MASX J15035031-7729407,15 03 49.75,-77 29 40.0
212,ESO 067-G005,15 04 35.35,-72 36 56.5
213,WKK 4603,15 05 29.10,-49 34 07.0


In [71]:
#Narrowing it down
table_values = new.as_matrix()
targets = [FixedTarget(coord=SkyCoord(ra = RA , dec = Dec,unit =(u.hourangle, u.deg)), name=Name) 
                                      for Name, RA, Dec in table_values]

observing_range = [observe_start, observe_end]

constraints = [AirmassConstraint(1.5)]
observing_table = observability_table(constraints, CTIO, targets, time_range=observing_range)

print(observing_table)


      target name       ever observable ... fraction of time observable
----------------------- --------------- ... ---------------------------
WKK 4098                           True ...          0.7619047619047619
2MASX J14381221-7239480            True ...          0.5714285714285714
2MASX J14382229-7240149            True ...          0.5714285714285714
WKK 4172                           True ...          0.7619047619047619
WKK 4260                           True ...          0.7619047619047619
WKK 4266                           True ...          0.7619047619047619
WKK 4551                           True ...          0.7619047619047619
2MASX J15035031-7729407            True ...         0.23809523809523808
ESO 067-G005                       True ...          0.5714285714285714
WKK 4603                           True ...          0.7619047619047619
